# In this notebook we use Pre-trained model which is specified as copy model and we train it on our own annotated images. This strategy helps when your own training data is limited 

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
%tensorflow_version 1.x

Mounted at /content/drive
TensorFlow 1.x selected.


In [2]:
!pip install tiffile
!pip install elasticdeform
!pip install keras==2.2.5
!pip install csbdeep
!pip install stardist
!pip install gputools
!pip install btrack

In [3]:
%cd '/content/drive/My Drive/SmartSeeds/'
import os
import glob
import sys
sys.path.append('../')
import glob
from Utils import SmartSeeds2D
from tifffile import imread, imwrite
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

/content/drive/My Drive/SmartSeeds


Using TensorFlow backend.


# In the cell below specify the following:

1) Directory where the training data is, inside this directory there should be the two subfolders called Raw and Mask. Inside the Raw folder are the raw images and inside the Mask folder are the labelled images.

2) The training data for doing UNET training is stored in NPZ format so please specify the NPZ filename which is suitable for your data.

3) Model directory is where the trained Neural network models are stored, please chooose a location if you want to change the default location which is where the training data is.

4) Copy Model name is optional, in case you have a previouis trained model and want to re-train it on new data but store it with a new name.

5) Model name is the unique name of the trained models.

In [4]:
Data_dir = '/content/drive/My Drive/FMIChallenge/'
NPZ_filename = 'FMINuclei'
Model_dir = Data_dir
Model_Name = 'FMINuclei'


# In this cell choose the network training parameters for the Neural Network

1) NetworkDepth = Depth of the network, with each increasing depth the image is downsampled by 2 hence the XY dimension of the data / 2^depth has to be greater than 1.

2) Epochs, training for longer epochs ensures a well converged network and requires longer GPU runtimes.

3) Learning rate is the parameter which controls the step size used in the optimization process and it should not be greater than 0.001 at the start of the training.

4) batch size controls the number of images used for doing stochastic gradient descent and is a parameter that is limited by the GPU RAM available, if you do not have a lot of ran batch size < 10 should be optimal. 

5) PatchX,Y is the patch size used for making patches out of the iamge data. The original image is broken down into patches for training. Patch size is chosen based on having enough context for the network to learn but at the same time not being too big to obscure memory usage.

6) Kernel is the receptive field of the neural network, usual choices are 3,5 or 7 but not larger than that. This is the size of the convolutional kernel used in the network

7) n_patches_per_image is the number of patches sampled for each image to create the npz file, choose an optimal value so that the file is not too big for the computer memory. 

8) Rays is the number of rays used the learn the distance map, low rays decreases the spatial resoultion and high rays are able to resolve the shape better.


9) OpenCL is a boolean parameter that is set true if you want to do some opencl computations on the GPU, this requires GPU tools but if you do not have them set this to false.

Some optimal values have been chosen by default and should work well for any NVDIA enabled GPU computer

In [5]:
#Network training parameters
NetworkDepth = 5
Epochs = 50
LearningRate = 1.0E-4
batch_size = 1
PatchX = 512
PatchY = 512
Kernel = 3
DownsampleFactor = 1
n_patches_per_image = 16
Rays = 128
use_gpu_opencl = True
GenerateNPZ = False
TrainUNET = True
TrainSTAR = True
grid = (2,2)

In [6]:

model = SmartSeeds2D(BaseDir = Data_dir, NPZfilename = NPZ_filename, model_name = Model_Name, model_dir = Model_dir, n_patches_per_image = n_patches_per_image, DownsampleFactor = DownsampleFactor, GenerateNPZ = GenerateNPZ,TrainUNET = TrainUNET, TrainSTAR = TrainSTAR, grid = grid, PatchX= PatchX, PatchY= PatchY,  use_gpu = use_gpu_opencl,  batch_size = batch_size, depth = NetworkDepth, kern_size = Kernel, n_rays = Rays, epochs = Epochs, learning_rate = LearningRate)

  0%|          | 0/12 [00:00<?, ?it/s]

Instance segmentation masks: 12
Semantic segmentation masks: 12
Training StarDistModel model


100%|██████████| 12/12 [00:00<00:00, 220.64it/s]

12
number of images:  12
- training:        10
- validation:       2
Configuration for a :class:`StarDist2D` model.

    Parameters
    ----------
    axes : str or None
        Axes of the input images.
    n_rays : int
        Number of radial directions for the star-convex polygon.
        Recommended to use a power of 2 (default: 32).
    n_channel_in : int
        Number of channels of given input image (default: 1).
    grid : (int,int)
        Subsampling factors (must be powers of 2) for each of the axes.
        Model will predict on a subsampled grid for increased efficiency and larger field of view.
    backbone : str
        Name of the neural network architecture to be used as backbone.
    kwargs : dict
        Overwrite (or add) configuration attributes (see below).


    Attributes
    ----------
    unet_n_depth : int
        Number of U-Net resolution levels (down/up-sampling layers).
    unet_kernel_size : (int,int)
        Convolution kernel size for all (U-Net) con


/usr/local/lib/python3.6/dist-packages/csbdeep/models/base_model.py:148: UserWarning: output path for model already exists, files may be overwritten: /content/drive/My Drive/FMIChallenge/FMINuclei
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


...

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Epoch 1/50
100/100 [==============================] - 44s 445ms/step - loss: 1.9666 - prob_loss: 0.4893 - dist_loss: 29.5463 - prob_kld: 0.2302 - dist_relevant_mae: 29.5463 - dist_relevant_mse: 1162.7620 - val_loss: 1.8858 - val_prob_loss: 0.3486 - val_dist_loss: 30.7447 - val_prob_kld: 0.0797 - val_dist_relevant_mae: 30.7446 - val_dist_relevant_mse: 1263.9186



Epoch 2/50
100/100 [==============================] - 39s 388ms/step - loss: 1.5868 - prob_loss: 0.3675 - dist_loss: 24.3855 - prob_kld: 0.1084 - dist_relevant_mae: 24.3847 - dist_relevant_mse: 886.5621 - val_loss: 1.2465 - val_prob_loss: 0.3799 - val_dist_loss: 17.3320 - val_prob_kld: 0.1110 - val_dist_relevant_mae: 17.3304 - val_dist_relevant_mse: 521.4189
Epoch 3/50
100/100 [==============================] - 42s 419ms/step - loss: 1.0125 - prob_loss: 0.3309 - dist_loss: 13.6325 - prob_kld: 0.0718 - dist_relevant_mae: 13.6303 - dist_relevant_mse: 322.5327 - val_loss: 0.9940 - val_prob_loss: 0.3268 - val_dist_loss: 13.3438 - val_prob_kld: 0.0579 - val_dist_relevant_mae: 13.3420 - val_dist_relevant_mse: 325.6061
Epoch 4/50
100/100 [==============================] - 42s 419ms/step - loss: 0.9044 - prob_loss: 0.2978 - dist_loss: 12.1310 - prob_kld: 0.0388 - dist_relevant_mae: 12.1293 - dist_relevant_mse: 266.9265 - val_loss: 0.9413 - val_prob_loss: 0.3097 - val_dist_loss: 12.6326 - val_p

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:08<00:02,  1.83it/s, 0.667 -> 0.833]


Using optimized values: prob_thresh=0.664585, nms_thresh=0.3.
Saving to 'thresholds.json'.
Training UNET model
number of training images:	 173
number of validation images:	 19
image size (2D):		 (512, 512)
axes:				 SYXC
channels in / out:		 1 / 1
Config(axes='YXC', n_channel_in=1, n_channel_out=1, n_dim=2, probabilistic=False, train_batch_size=1, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_epochs=50, train_learning_rate=0.0001, train_loss='mae', train_reduce_lr={'patience': 5, 'factor': 0.5}, train_steps_per_epoch=400, train_tensorboard=True, unet_input_shape=(None, None, 1), unet_kern_size=3, unet_last_activation='linear', unet_n_depth=5, unet_n_first=48, unet_residual=True)
Epoch 1/50
400/400 [==============================] - 30s 74ms/step - loss: 0.1594 - mse: 0.1005 - mae: 0.1594 - val_loss: 0.0900 - val_mse: 0.0598 - val_mae: 0.0900
Epoch 2/50
400/400 [==============================] - 27s 68ms/step 